<a href="https://colab.research.google.com/github/dhruvilk/CS301-Project/blob/master/Jax_Implementation_of_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download Religious-and-philosophical-texts

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
from google.colab import files 
files.upload()

Saving pg2680.txt to pg2680 (1).txt


{'pg2680.txt': b'\xef\xbb\xbfThe Project Gutenberg EBook of Meditations, by Marcus Aurelius\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Meditations\r\n\r\nAuthor: Marcus Aurelius\r\n\r\nPosting Date: December 25, 2008 [EBook #2680]\r\nRelease Date: June, 2001\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK MEDITATIONS ***\r\n\r\n\r\n\r\n\r\nProduced by J. Boulton\r\n\r\n\r\n\r\n\r\n\r\nMEDITATIONS\r\n\r\nBy Marcus Aurelius\r\n\r\n\r\n\r\n\r\nCONTENTS\r\n\r\n\r\n     NOTES\r\n\r\n     INTRODUCTION\r\n\r\n     FIRST BOOK\r\n\r\n     SECOND BOOK\r\n\r\n     THIRD BOOK\r\n\r\n     FOURTH BOOK\r\n\r\n     FIFTH BOOK\r\n\r\n     SIXTH BOOK\r\n\r\n     SEVENTH BOOK\r\n\r\n     EIGHTH BOOK\r\n\r\n     NINTH BOOK\r\n\r\n     TENTH

In [ ]:
import numpy as np
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
from jax.nn import softmax, one_hot

with open(r'pg2680.txt') as fp:
    corpus = fp.readlines()
fp.close()

list = corpus

i = 0
for sentence in list:
    newStr = ""
    for word in sentence:
        if word.isalnum():
            newStr = newStr + word
        elif word == ' ':
            newStr = newStr + word

    newStr = newStr.lower()
    list[i] = newStr
    i = i + 1

while "" in list:
    list.remove("")

list = [x.strip(' ') for x in list]

def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

def create_vocabulary(tokenized_corpus):
    vocab = {}
    vocab['<PAD>'] = 0
    i = 1
    for sentence in tokenized_corpus:
        for word in sentence:
            if word != vocab:
                vocab[word] = i
            i += 1

    return vocab


def generate_train_data(tokenized_corpus, vocab, window_size):
    X = []
    y = []
    for phrase in tokenized_corpus:
        for i in range(len(phrase)):
            target = i
            context = []

    for j in range(-window_size, window_size + 1):
        if j == 0:
            continue
        try:
            if i + j < 0:
                context.append('<PAD>')
                continue
            context.append(phrase[i + j])
        except Exception:
            context.append('<PAD>')
    X.append([vocab[k] for k in context])
    y.append(vocab[phrase[target]])

    return jnp.array(X), jnp.array(y)

window_size = 2
tokenized_corpus = tokenize_corpus(list)
vocab = create_vocabulary(tokenized_corpus)
X, y = generate_train_data(tokenized_corpus, vocab, window_size)

class Word2VecCBOW():
  def __init__(self, window_size, embed_dim, vocab_size, random_state):
        # Defines the key to be used for the random creation of the weights
        self.key = random.PRNGKey(random_state)
        self.vocab_size = vocab_size
        self.linear = self._create_random_matrix(vocab_size, embed_dim)
        self.soft = self._create_random_matrix(embed_dim, vocab_size)
        
        # Vectorizes the predict method
        self.predict = vmap(self._predict, in_axes=(None, 0))
    
  def train(self, X, y, num_epochs, batch_size):
        y = one_hot(y, self.vocab_size)
        X = one_hot(X, self.vocab_size)
        params = [self.linear, self.soft]
        for epoch in range(num_epochs):
            print(f'Epoch: {epoch}')
            for X_batch, y_batch in self.generate_batches(X, y, batch_size):
                    params = self.update(params, X_batch, y_batch)
            print(f'Loss: {float(self.l.primal)}')
        self.linear = params[0]
        self.soft = params[1]
  def _predict(self, params, X):
        activations = []
        for x in X:
            activations.append(jnp.dot(x, params[0]))
# Averages the activations
        activation = jnp.mean(jnp.array(activations), axis=0)
        logits = jnp.dot(activation, params[1])
        result = softmax(logits)
        
        return result
    
  def _create_random_matrix(self, window_size, embed_dim):
        w_key = random.split(self.key, num=1)
        
        return 0.2 * random.normal(self.key, (window_size, embed_dim))
  def loss(self, params, X, y):
        preds = self.predict(params, X)
        l = -jnp.mean(preds * y)
        self.l = l
        return l
  def update(self, params, X, y, step_size=0.02):
        grads = grad(self.loss)(params, X, y)
        return [params[0] - step_size * grads[0],
                params[1] - step_size * grads[1]]
    
  def get_embedding(self):
        return self.linear
  def generate_batches(self, X, y, batch_size):
        for index, offset in enumerate(range(0, len(y), batch_size)):
            yield X[offset: offset + batch_size], y[offset: offset + batch_size]

w2v = Word2VecCBOW(2, 32, len(vocab), 42)
w2v.train(X, y, 10, 32)
w2v.get_embedding()

Epoch: 0
Loss: -0.0
Epoch: 1
Loss: -0.0
Epoch: 2
Loss: -0.0
Epoch: 3
Loss: -0.0
Epoch: 4
Loss: -0.0
Epoch: 5
Loss: -0.0
Epoch: 6
Loss: -0.0
Epoch: 7
Loss: -0.0
Epoch: 8
Loss: -0.0
Epoch: 9
Loss: -0.0


DeviceArray([[-0.17953794,  0.03117072,  0.21723711, ..., -0.17067401,
               0.15227185, -0.10445432],
             [ 0.09183712,  0.24994348,  0.28888342, ..., -0.2809554 ,
              -0.19657919,  0.37955734],
             [ 0.07019374, -0.1045405 ,  0.03563719, ...,  0.3601839 ,
               0.14363597,  0.05926179],
             ...,
             [ 0.10087724,  0.01041753, -0.07444171, ...,  0.3051966 ,
               0.30342498, -0.0908545 ],
             [-0.15038489,  0.0223883 , -0.08902696, ..., -0.0343544 ,
               0.02924858,  0.22984397],
             [-0.08138925,  0.06099118,  0.071344  , ..., -0.1868572 ,
              -0.00401894, -0.12961282]], dtype=float32)